# Evolutionary Camouflage Versus a Learning Predator Population

---

**EvoCamoVsLearnPredPop.ipynb**

August 23, 2022: this version runs “local mode” with both predator and prey running on the same machine.

(The former behavior available with `Rube_Goldberg_mode = True`)

In [1]:
# "Rube Goldberg" mode refers to running camouflage evolution on my laptop while
# running predator vision in cloud via Colab. State is passed back and forth via
# files on Google Drive.

# TODO 20220822
# Rube_Goldberg_mode = True
Rube_Goldberg_mode = False

def if_RG_mode(for_RG_mode, for_normal_mode):
    return for_RG_mode if Rube_Goldberg_mode else for_normal_mode

# PredatorEye directory on Drive.
pe_directory = '/content/drive/My Drive/PredatorEye/'

# Shared "communication" directory on Drive.
shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
                              '/Users/cwr/camo_data/comms/')

# This was meant (20220716) to allow reading original pre-trained model from
# Google Drive, but I'll need to retrain it for M1 (Apple Silicon).
g_drive_pe_dir = ('/Users/cwr/Library/CloudStorage/' +
                  'GoogleDrive-craig.w.reynolds@gmail.com/' +
                  'My Drive/PredatorEye/')

# Directory for pre-trained Keras/TensorFlow models.
saved_model_directory = if_RG_mode(pe_directory, g_drive_pe_dir) + 'saved_models/'


print('Rube_Goldberg_mode =', Rube_Goldberg_mode)
print('shared_directory =', shared_directory)
print('saved_model_directory =', saved_model_directory)

# Pathname of pre-trained Keras/TensorFlow model
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'

# Directory on Drive for storing fine-tuning dataset.
fine_tuning_directory = shared_directory + 'fine_tuning/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from pathlib import Path

from tensorflow import keras
import numpy as np
import random
import math

# %tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
if Rube_Goldberg_mode:
    sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
else:
    sys.path.append('/Users/cwr/Documents/code/PredatorEye/')
import DiskFind as df

Rube_Goldberg_mode = False
shared_directory = /Users/cwr/camo_data/comms/
saved_model_directory = /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/
TensorFlow version: 2.9.2


# Ad hoc “predator server”

In [2]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        perform_step(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def perform_step(step, directory):
    ############################################################################
    # TODO 20220909 add verbose "start of step" logging -- getting hard to read.
    print()
    print('step', step)
    print()
    ############################################################################
    wait_for_reply(step, shared_directory)
    write_response_file(step, shared_directory)
    delete_find_file(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
# (TODO 20220911 this has grown huge, needs to be refactored into smaller parts)
def write_response_file(step, directory):
    # Read image file and check for expected format.
    image_pathname = make_camo_pathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    # TODO 20220907 can/should this TF version replace pixel_tensor below?
    #               no this causes an error in Predator.fine_tune_model()
    tf_pixel_tensor = tf.convert_to_tensor([pixel_tensor])
    
    ############################################################################
    # TODO 20220911 testing new Tournament class

#     # Choose a random tournament of three from population of Predators.
#     tournament = Predator.choose_tournament()

#     # Collect predictions from each Predator: where in image is prey located?
#     predictions_xy3 = [predator.model.predict(tf_pixel_tensor)[0]
#                        for predator in tournament]
    ############################################################################

    # Collect self-supervision signal from TexSyn: positions of prey centers.
    prey_centers_xy3 = read_3_centers_from_file(step, directory)

    ############################################################################
    # TODO 20220911 testing new Tournament class

#     # Sort predictions from the three Predators in a tournament, according to
#     # ”accuracy” (least aim error).
#     ranked_predictions_xy3 = rank_tournament(predictions_xy3, prey_centers_xy3)
    
#     ttt = Tournament(pixel_tensor, prey_centers_xy3)
    tournament = Tournament(pixel_tensor, prey_centers_xy3)


    # Write response file (contains 3 xy positions as 6 floats on [0,1]).
#     response_string = xy3_to_str(ranked_predictions_xy3)
#     response_string = xy3_to_str(ttt.ranked_predictions())
    response_string = xy3_to_str(tournament.ranked_predictions())


    ############################################################################
    verify_comms_directory_reachable()
    with open(make_find_pathname(step, directory), 'w') as file:
        file.write(response_string)
    print('Wrote ' + "'" + response_string + "'",
          'to response file', Path(make_find_pathname(step, directory)).name)
    
    # Merge this step's image into fine-tuning dataset, and related bookkeeping.
    ############################################################################
    # TODO 20220911 testing new Tournament class
#     best_prediction = ranked_predictions_xy3[0]
#     best_prediction = ttt.ranked_predictions()[0]
    best_prediction = tournament.ranked_predictions()[0]
    ############################################################################
    fine_tuning_dataset.update(pixel_tensor, best_prediction, step, directory)

    # Fine-tune models of each Predator in tournament.
    ############################################################################
    # TODO 20220911 testing new Tournament class
#     for i in range(len(tournament)):
#     for i in range(ttt.size()):
#         # TODO oh this has to be inside Tournament
#         tournament[i].fine_tune_model(pixel_tensor, predictions_xy3[i], step, directory)
    
    # TODO 20220911 this really sucks
    for p in Predator.population:
        p.set_step(step)
    
#     ttt.fine_tune_models()
    tournament.fine_tune_models()

    ############################################################################

        
    ############################################################################
    # TODO 20220913 update population if needed at end of Tournament
    tournament.update_predator_population()
    ############################################################################



# Delete the given file, presumably after having written the next one.
def delete_find_file(step, directory):
    # Why doesn't pathlib provide a Path.remove() method like os?
    # TODO oh, missing_ok was added at pathlib version 3.8.
    # Path(makeMyPathname(step, directory)).unlink(missing_ok=True)
    p = Path(make_find_pathname(step, directory))
    if p.exists():
        p.unlink()

# Delete any remaining file in commuications directory to start a new run.
def clean_up_communication_directory():
    def delete_directory_contents(directory_path):
        for path in directory_path.iterdir():
            print('Removing from communication directory:', path)
            if path.is_dir():
                delete_directory_contents(path)
                path.rmdir()
            else:
                path.unlink()
    delete_directory_contents(Path(shared_directory))

# From pathname for file of given step number from the "other" agent.
def make_camo_pathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for "find_xx.txt" response file from "this" agent.
def make_find_pathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Form pathname for "prey_xx.txt" ground truth file from "other" agent.
def make_prey_pathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'

# Used to ping the comms directory when it seems hung.
def write_ping_file(count, step, directory):
    pn = directory + 'ping_cloud_' + str(step) + '.txt'
    verify_comms_directory_reachable()
    with open(pn, 'w') as file:
        file.write(str(count))
    print('Ping comms: ', count, pn)

# Wait until other agent's file for given step appears.
def wait_for_reply(step, directory):
    camo_pathname = Path(make_camo_pathname(step, directory))
    camo_filename = camo_pathname.name
    prey_pathname = Path(make_prey_pathname(step, directory))
    prey_filename = prey_pathname.name
    print('Waiting for', camo_filename, 'and', prey_filename, '...',
          end='', flush=True)
    start_time = time.time()
    # Loop until both files are present, waiting 1 second between tests.
    test_count = 0
    while not (is_file_present(camo_pathname) and
               is_file_present(prey_pathname)):
        time.sleep(1)
        test_count += 1
        if (test_count % 100) == 0:
            write_ping_file(test_count, step, directory)
    print(' done, elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists()
def is_file_present(file):
    result = False
    verify_comms_directory_reachable()
    filename = Path(file).name
    directory = Path(file).parent
    for i in directory.iterdir():
        if i.name == filename:
            result = True
    return result

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in Path(directory).iterdir():
        name = filename.name
        if other_prefix == name[0:len(other_prefix)]:
            steps.append(int(name.split(".")[0].split("_")[1]))
    return max(steps)

# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    # Read contents of file as string.
    verify_comms_directory_reachable()
    with open(make_prey_pathname(step, directory), 'r') as file:
        prey_centers_string = file.read()
    # Split string at whitespace, map to 6 floats, reshape into 3 xy pairs.
    # (TODO could probably be rewritten cleaner with "list comprehension")
    array = np.reshape(list(map(float, prey_centers_string.split())), (3, 2))
    return array.tolist()

# Keep log of in_disk metric.
def write_in_disk_log(step, history):
    if step % 10 == 0:
        in_disk = history.history["in_disk"][0]
        pathname = shared_directory + 'in_disk_log.csv'
        verify_comms_directory_reachable()
        with open(pathname, 'a') as file:
            if step == 0:
                file.write('step,in_disk\n')
            file.write(str(step) + ',' + "{:.4f}".format(in_disk) + '\n')

# Just wait in retry loop if shared "comms" directory become unreachable.
# Probably will return shortly, better to wait than signal a file error.
# (This is called from places with a local "directory" but it uses global value.)
def verify_comms_directory_reachable():
    seconds = 0
    # shared_directory_pathname = Path(shared_directory)
    # while not shared_directory_pathname.is_dir():
    while not Path(shared_directory).is_dir():
        print("Shared “comms” directory,", shared_directory, 
              "has been inaccessible for", seconds, "seconds.")
        time.sleep(1)  # wait 1 sec
        seconds += 1

# Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
def sort_xy3_by_proximity_to_point(xy3, point):
    # print('xy3 =', xy3)
    xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
    # print('xy3_plus_distance =', xy3_plus_distance)
    sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
    # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
    sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
    # print('sorted_xy3 =', sorted_xy3)
    return sorted_xy3

# Convert xy3 to string: [[x,y], [p,q], [r,s]] -> 'x y p q r s '
def xy3_to_str(xy3):
    return ''.join('%s ' % i for i in flatten_nested_list(xy3))

def flatten_nested_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Read pre-trained model

As I integrate this into the Predator class, this is no longer “Read pre-trained model” but more like “Some utilities for reading the pre-trained model”

In [3]:
# Read pre-trained TensorFlow "predator vision" model.

# print('Reading pre-trained model from:', trained_model)

# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

def read_default_pre_trained_model():
    print('Reading pre-trained model from:', trained_model)
    return keras.models.load_model(trained_model, custom_objects=dependencies)

# FineTuningDataset

In [4]:
class FineTuningDataset:
    """Manages the dataset of images and labels for fine-tuning."""

    # Accumulated a new “training set” of the most recent N steps seen so far. (See
    # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
    # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
    # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
    max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
    # List of "pixel tensors".
    fine_tune_images = []
    # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
    fine_tune_labels = []

    def update(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # Accumulate the most recent "max_training_set_size" training samples.
        self.fine_tune_images.append(pixel_tensor)
        self.fine_tune_labels.append(sorted_xy3)

        # If training set has become too large, slice off first element of each.
        if len(self.fine_tune_images) > self.max_training_set_size:
            self.fine_tune_images = self.fine_tune_images[1:]
            self.fine_tune_labels = self.fine_tune_labels[1:]

        print('  fine_tune_images shape =', np.shape(self.fine_tune_images),
              '-- fine_tune_labels shape =', np.shape(self.fine_tune_labels))
        

# Create a global FineTuningDataset object.
# (TODO globals are usually a bad idea, reconsider this.)
fine_tuning_dataset = FineTuningDataset()

# Predator class

In [5]:
class Predator:
    """Represents a Predator in the camouflage simulation. It has a CNN-based
       model of visual hunting that identified the position of likely prey."""

    # Global list of active Predators. (As a class variable.)
    population = []
    
    ############################################################################
    # TODO 20220905 cache default_pre_trained_model
    default_pre_trained_model = read_default_pre_trained_model()
    ############################################################################


    def __init__(self):
        # Each Predator has its own copy of a prey-finding CNN-based model.
        self.model = []
        # By default add this new Predator to the population (TODO reconsider)
        Predator.population.append(self)
        ########################################################################
        # TODO 20220909
        # keep history of predation events: was hunt a success or a failure?
        self.successes = []
        ########################################################################



    ############################################################################
    # TODO 20220905 cache default_pre_trained_model
    
#     # Set this Predator's model to default.
#     def initialize_to_pre_trained_model(self):
#         self.model = read_default_pre_trained_model()

#     # Set this Predator's model to default.
#     def initialize_to_pre_trained_model(self):
#         self.model = self.default_pre_trained_model

#     # TODO 20220905 changed back because was just copying pointer not deep copy.
#     def initialize_to_pre_trained_model(self):
#         self.model = read_default_pre_trained_model()
    ############################################################################

    # Keep track of how often selected prey is nearest center:
    nearest_center = 0
    
    # TODO 20220911 this is a goofy hack to avoid passing the "step" parameter
    # to Predator.fine_tune_model() for the sole purpose of logging about
    # "nearest_center" which really needs to be redesigned.
    def set_step(self, step):
        self.step = step

    # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
    # training set. Assume they were "near misses" and so training label is actual
    # (ground truth) center of disk nearest prediction. Keep a max number of old
    # steps to allow gradually forgetting the earliest part of the run.
    
#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789

    ############################################################################
    # TODO 20220911 remove dependency of Predator.fine_tune_model() on step and
    #               directory args. replace them with prey_centers_xy3
#     def fine_tune_model(self, pixel_tensor, prediction, step, directory):
    def fine_tune_model(self, pixel_tensor, prediction, prey_centers_xy3):

        # Assume the predator was "aiming for" that one but missed by a bit.
#         xy3 = read_3_centers_from_file(step, directory)
#         sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)
        sorted_xy3 = sort_xy3_by_proximity_to_point(prey_centers_xy3, prediction)


        # keep track of how often selected prey is nearest center:
#         temp = xy3.copy()  # needed? (much later 20220911, no I don't think so)
        temp = prey_centers_xy3.copy()  # needed? (much later 20220911, no I don't think so)

        sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
        if sorted_by_dist_to_center[0] == sorted_xy3[0]:
            Predator.nearest_center += 1
        
        # TODO 20220829 "ad-hoc-ly" adjust for two models running, hence NC count doubled.
        # print('  nearest_center:',
        #       str(int(100 * float(self.nearest_center) / (step + 1))) + '%',
        #       '(nearest_center =', self.nearest_center, ', steps =', step + 1, ')')
        nc = self.nearest_center / 2
#         print('  nearest_center:',
#               str(int(100 * float(nc) / (step + 1))) + '%',
#               '(nearest_center =', nc, ', steps =', step + 1, ')')
        print('  nearest_center:',
              str(int(100 * float(nc) / (self.step + 1))) + '%',
              '(nearest_center =', nc, ', steps =', self.step + 1, ')')

        ########################################################################


        # Convert training data list to np arrays
        images_array = np.array(fine_tuning_dataset.fine_tune_images)
        labels_array = np.array([x[0] for x in fine_tuning_dataset.fine_tune_labels])

        # print('images_array.shape =', images_array.shape,
        #       '-- labels_array.shape =', labels_array.shape)

    	# Skip fine-tuning until dataset is large enough (10% of max size).
        ########################################################################
        # TODO 20220803 clone model

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # TODO 20220823 restore "Skip fine-tuning until dataset is large enough"
        
#         print('disabled "Skip fine-tuning until dataset is large enough"')
#         if images_array.shape[0] > 0: ##########################################

        if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):

            # # Do fine-tuning training step using data accumulated during run.
            # history = self.model.fit(x=images_array, y=labels_array)
            
            # TODO 20220823 -- run fine-tuning on CPU only.
            print('Running on CPU ONLY!')
            with tf.device('/cpu:0'):
                # Do fine-tuning training step using data accumulated during run.
                history = self.model.fit(x=images_array, y=labels_array)
            
            # Keep log of in_disk metric:
#             write_in_disk_log(step, history)
            write_in_disk_log(self.step, history)


        print(end='  ')
        self.print_model_trace()
        ########################################################################
        
        
        # TODO 20220911 can this be moved to end of fine_tune_model()?
#         for i in range(3):
#              tournament[i].record_predation_success(predictions_xy3[i], prey_centers_xy3[i])

#         # TODO 20220911 moved this here from write_response_file(), are these the correct args?
#         self.record_predation_success(prediction, xy3)

        self.record_predation_success(prediction, prey_centers_xy3)


    ############################################################################



    
    ############################################################################
    # TODO 20220803 clone model

    # # Copy the neural net model of a given predator into this one.
    # def copy_model(self, another_predator):
    #     # No this is wrong, just does a shallow copy
    #     # self.model = another_predator.model
    #     self.model = tf.keras.models.clone_model(another_predator.model)
    #     # Compile newly cloned model.
    #     # compile_disk_finder_model(self.model)
    #     df.compile_disk_finder_model(self.model)

#     # Copy the neural net model of a given predator into this one. (From "Make
#     # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
#     def copy_model(self, another_predator):
#         other_model = another_predator.model
#         # Clone layer structure of other model.
#         # self.model = tf.keras.models.clone_model(another_predator.model)
#         self.model = tf.keras.models.clone_model(other_model)
#         # Compile newly cloned model.
#         df.compile_disk_finder_model(self.model)

#         # Copy weights of other model.
#         self.model.set_weights(other_model.get_weights())


    # TODO 20220907 refactor into two methods

    # Copy the neural net model of a given predator into this one. (From "Make
    # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
    
    def copy_model_of_another_predator(self, another_predator):
        self.copy_model(another_predator.model)

    
#     def copy_model(self, another_predator):
#         other_model = another_predator.model
    def copy_model(self, other_model):
        # Clone layer structure of other model.
        self.model = tf.keras.models.clone_model(other_model)
        # Compile newly cloned model.
        df.compile_disk_finder_model(self.model)
        # Copy weights of other model.
        self.model.set_weights(other_model.get_weights())

    ############################################################################

    # Modify this Predator's model by adding noise to its weights.
#     def jiggle_model(self):
#         weight_perturbation(self.model, 0.001)
############################################################################
# TODO 20220909 trying to control TF retracing
#     def jiggle_model(self, strength = 0.001):
#         weight_perturbation(self.model, strength)
    def jiggle_model(self, strength = 0.001):
        weight_perturbation(self.model, tf.constant(strength))
############################################################################


    
    # Print the "first" weight of each layer of this Predator's Keras model.
    def print_model_trace(self):
        for layer in self.model.layers:
            trainable_weights = layer.trainable_variables
            for weight in trainable_weights:            
                weight_shape = tf.shape(weight)
                total_size = tf.math.reduce_prod(weight_shape)
                reshape_1d = tf.reshape(weight, [total_size])
                
                # TODO 20220829 take "middle" not first parameter.
                # value = reshape_1d[0].numpy()
                middle = math.floor(total_size / 2)
                value = reshape_1d[middle].numpy()
        
                # print(round(value, 2), end = " ")
                print(round(value, 3), end = " ")

        print()

    ############################################################################
    # TODO 20220905
    # Create the given number of Predator objects
    # (TODO maybe the pretrained model should be an arg?)
    def initialize_predator_population(population_size):
        for i in range(population_size):
            p = Predator()
            print('Predator instance address:', id(p))

#             p.initialize_to_pre_trained_model()
            # TODO 20220907 maybe it sould just do this by default in constructor?
            p.copy_model(Predator.default_pre_trained_model)
        
            p.print_model_trace()


            
#             p.jiggle_model()
#             p.jiggle_model(0.1)
#             p.jiggle_model(0.01)
#             p.jiggle_model(0.001)
            p.jiggle_model(0.003)




            p.print_model_trace()


            # does this help?
            # Compile newly cloned model.
            df.compile_disk_finder_model(p.model)

            p.print_model_trace()
        print('Created population of', population_size, 'Predators.')
    ############################################################################
    
    ######################################################################################
#     # TODO 20220907 prototype tournament maker, eventually random, now just first 3
#     def choose_tournament(size = 3):
#         tournament = []
#         for i in range(size):
#             tournament.append(Predator.population[i])
#         return tournament

    # TODO 20220909 do real random selection of tournament from population

    # TODO 20220907 prototype tournament maker, eventually random, now just first 3
    def choose_tournament(size = 3):
#         tournament = []
#         for i in range(size):
#             tournament.append(Predator.population[i])
        assert len(Predator.population) >= size, "Population smaller than tournament size."
        return random.sample(Predator.population, size)


    ######################################################################################

    
#         ######################################################################################
#         # TODO 20220909 keep history of predation events: was hunt a success or a failure?
#     #     # Given one predator's xy prediction, and three prey center positions, find the
#     #     # "aim error": the distance from the predition to the nearest prey center.
#     #     def aim_error(prediction_xy, prey_centers_xy3):
#
#     #     def record_predation_success(self, prediction_xy, prey_centers_xy3):
#     # #         success = 0
#     #         ae = aim_error(prediction_xy, prey_centers_xy3)
#     # #         if ae < prey_radius:
#     # #             success = 1
#     #         variable = 0 if ae < prey_radius else 1
#     #         successes.append(success)
#
#         def record_predation_success(self, prediction_xy, prey_centers_xy3):
#             radius = df.relative_disk_radius()
#             distance = aim_error(prediction_xy, prey_centers_xy3)
#             self.successes.append(0 if distance < radius else 1)
#             self.successes = self.successes[-10:] # Trim to just last 10 of history
#     #         print('successes =', self.successes)
#     #         starve = (self.successes[-1] + self.successes[-2]) == 0
#             starve = ((len(self.successes) >= 2) and
#                       ((self.successes[-1] + self.successes[-2]) == 0))
#             print('successes =', self.successes, "starve!" if starve else "")
#         ######################################################################################


    ############################################################################
    # TODO 20220911 update population if needed at end of Tournament
    
    success_history_max_length = 10
    
    def record_predation_success(self, prediction_xy, prey_centers_xy3):
        radius = df.relative_disk_radius()
        distance = aim_error(prediction_xy, prey_centers_xy3)
        self.successes.append(0 if distance < radius else 1)
        
#         self.successes = self.successes[-10:] # Trim to just last 10 of history
        # Trim to max length.
        self.successes = self.successes[-self.success_history_max_length:]

#         starve = ((len(self.successes) >= 2) and
#                   ((self.successes[-1] + self.successes[-2]) == 0))
#         print('successes =', self.successes, "starve!" if starve else "")

    # Defines starvation as suceeding less than half of the preceeding hunts.
    def starvation(self):
        starving = False
        if len(self.successes) == self.success_history_max_length:
            count = sum(self.successes)
            if count < self.success_history_max_length / 2:
                starving = True
                
#         if starving:
#             print(self.successes, '  starving!!')
#         else:
#             print(self.successes)

        return starving

    ############################################################################

    
    ############################################################################
    # TODO 20220913 replace starved Predator with new one.
    def replace_in_population(self, parent):
        self.copy_model_of_another_predator(parent)
        self.jiggle_model(0.003)
        print('reinitializing predator', id(self))
    ############################################################################




# Utility based on https://stackoverflow.com/a/64542651/1991373
# TODO 20220907 added this to avoid always getting the same random_weights
# @tf.function

@tf.function(reduce_retracing=True)
def weight_perturbation(model, max_range):
    """Add noise to all weights in a Keras model."""
    for layer in model.layers:
        trainable_weights = layer.trainable_variables
        for weight in trainable_weights:
            random_weights = tf.random.uniform(tf.shape(weight),
                                               max_range / -2,
                                               max_range / 2,
                                               dtype=tf.float32)
            weight.assign_add(random_weights)

            
# print(tf.random.uniform((2, 2), 1 / -2, 1 / 2, dtype=tf.float32))
# print(tf.random.uniform((2, 2), 1 / -2, 1 / 2, dtype=tf.float32))
# print()

# TODO 20220905 -- this should probably not be at global scope:
# Create population of 12 Predators.
Predator.initialize_predator_population(12)
# Predator.initialize_predator_population(3)
print('population size:', len(Predator.population))

Reading pre-trained model from: /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4
Predator instance address: 6101265952
-0.391 -0.008 -0.108 -0.969 -0.191 -0.095 0.104 0.01 -0.005 -3.86 0.29 1.045 -0.161 -5.435 -0.02 6.254 0.028 0.211 
-0.392 -0.009 -0.107 -0.97 -0.192 -0.094 0.105 0.008 -0.006 -3.858 0.289 1.047 -0.162 -5.436 -0.02 6.254 0.028 0.211 
-0.392 -0.009 -0.107 -0.97 -0.192 -0.094 0.105 0.008 -0.006 -3.858 0.289 1.047 -0.162 -5.436 -0.02 6.254 0.028 0.211 
Predator instance address: 6343461424
-0.391 -0.008 -0.108 -0.969 -0.191 -0.095 0.104 0.01 -0.005 -3.86 0.29 1.045 -0.161 -5.435 -0.02 6.254 0.028 0.211 
-0.391 -0.008 -0.108 -0.97 -0.192 -0.096 0.104 0.011 -0.004 -3.86 0.291 1.044 -0.162 -5.436 -0.019 6.253 0.028 0.209 
-0.391 -0.008 -0.108 -0.97 -0.192 -0.096 0.104 0.011 -0.004 -3.86 0.291 1.044 -0.162 -5.436 -0.019 6.253 0.028 0.209 
Predator instance address: 6322925632
-0.391 -0.008 -0.108 -

In [6]:
# foo = [1, 2, 3, 4, 5, 6]
# foo = [1, 2, 3]

# bar = foo[5:]
# baz = foo[-5:]

# print('foo', foo)
# print('bar', bar)
# print('baz', baz)

random.sample([2,4,6,8,10], 3)

[10, 6, 4]

# Prototype rank tournament of Predators by min prediction-prey distance 

In [7]:
#     # Order a tournamant of 3 Predators by sorting their three xy predictions. They
#     # are ordered by distance from prediction to the nearest of three prey centers
#     # (smaller being earlier in sort order -- think of it as "aiming error").
#     # TODO 20220913 I think this is obsolete now.
#     def rank_tournament(predictions_xy3, prey_centers_xy3):
#         return sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3)

#     # Given 3 predator prediction positions ("xy3"), sort them by "least aim error",
#     # that is, the distance from the prediction to the nearest prey position, so
#     # smallest errors sort to the front of the collection.
#     # TODO 20220913 I think this is obsolete now.
#     def sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3):
#         predictions_with_key = [[aim_error(xy, prey_centers_xy3), xy]
#                                 for xy in predictions_xy3]
#         # print('predictions_with_key =', predictions_with_key)
#         sorted_predictions_with_key = sorted(predictions_with_key,
#                                              key=lambda x: x[0])
#         # print('sorted_predictions_with_key =', sorted_predictions_with_key)
#         sorted_predictions = [x[1] for x in sorted_predictions_with_key]
#         # print('sorted_predictions =', sorted_predictions)
#         return sorted_predictions

# Given one predator's xy prediction, and three prey center positions, find the
# "aim error": the distance from the predition to the nearest prey center.
# TODO 20220913 maybe move this to inside Tournament class?
# TODO Oh, it is used in Predator.record_predation_success()
def aim_error(prediction_xy, prey_centers_xy3):
    min_aim_error = math.inf
    for xy in prey_centers_xy3:
        distance = df.dist2d(xy, prediction_xy)
        if min_aim_error > distance:
            min_aim_error = distance
    return min_aim_error


# rank_tournament([[0.2, 0.5], [0.4, 0.5], [0.3, 0.5]],
#                 [[0.5, 0.5], [0.6, 0.5], [0.7, 0.5]])

# Predator class

In [8]:
class Tournament:
    """Represents three Predators in a Tournament of the camouflage simulation."""
    
    class Member:
        """One Predator in a Tournament of the camouflage simulation."""
        def __init__(self, predator, tf_pixel_tensor, prey_centers_xy3):
            self.predator = predator
            self.tf_pixel_tensor = tf_pixel_tensor
            self.prey_centers_xy3 = prey_centers_xy3
            self.prediction = self.predator.model.predict(tf_pixel_tensor)[0]
            self.aim_error = aim_error(self.prediction, self.prey_centers_xy3)

    def __init__(self, pixel_tensor, prey_centers_xy3):
        # Store pixel data from current input image from TexSyn side.
        self.pixel_tensor = pixel_tensor
        # Also store it as a TF-style tensor        
        # TODO 20220907 should eventually replace the non-TF data.
        #               now causes an error in Predator.fine_tune_model()
        self.tf_pixel_tensor = tf.convert_to_tensor([self.pixel_tensor])
        # Store the positions of all prey centers as xy on image.
        self.prey_centers_xy3 = prey_centers_xy3
        # Choose random tournament of three from population of Predators.
        # Build a list with each wrapped in Tournament Member objects.
        self.members = [self.Member(predator, self.tf_pixel_tensor, prey_centers_xy3)
                        for predator in Predator.choose_tournament(3)]
        # Rank members by aim_error (smallest first)
        self.members = sorted(self.members, key=lambda m: m.aim_error)
        # Sort predictions from the three Predators in a tournament, according to
        # ”accuracy” (least aim error).
        # TODO should this be computed on the fly in Tournament.ranked_predictions()?
        self.ranked_predictions_xy3 = [member.prediction for member in self.members]

    # Gets the list of 3 prey center predictions, ranked most accurate at front.
    def ranked_predictions(self):
        return self.ranked_predictions_xy3
    
    # Perform fine-tuning step on each Predator in this Tournament.
    def fine_tune_models(self):        
        for member in self.members:
            member.predator.fine_tune_model(self.pixel_tensor,
                                            member.prediction,
                                            self.prey_centers_xy3)
            
    ############################################################################
    # TODO 20220913 update population if needed at end of Tournament
    def update_predator_population(self): 
        predator = self.members[-1].predator
        if predator.starvation():
            global xxx_temp_starvation_count
            xxx_temp_starvation_count += 1
            print()
            print('starving!!  ',
                  xxx_temp_starvation_count, ', ',
                  predator.step / xxx_temp_starvation_count, ', ',
                  xxx_temp_starvation_count / predator.step)
            
            # TODO 20220913 replace starved Predator with new one.
            parent = self.members[0].predator  # just choose the first/best for now
            predator.replace_in_population(parent)
            
            print()
    ############################################################################
    
xxx_temp_starvation_count = 0
    
#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789

# Run test

In [9]:
# # TODO 20220827 testing print_model_trace
# test_predator.print_model_trace()



# Keep track of how often selected prey is nearest center:
Predator.nearest_center = 0

# Predator.population = []
# TODO maybe a Predator.reset() method?

# Flush out obsolete files in comms directory.
clean_up_communication_directory()

# Start fresh run defaulting to step 0.
start_run()

Removing from communication directory: /Users/cwr/camo_data/comms/prey_26.txt
Removing from communication directory: /Users/cwr/camo_data/comms/prey_27.txt
Removing from communication directory: /Users/cwr/camo_data/comms/camo_27.png
Removing from communication directory: /Users/cwr/camo_data/comms/find_25.txt
Removing from communication directory: /Users/cwr/camo_data/comms/find_26.txt
Start run in /Users/cwr/camo_data/comms/

step 0

Waiting for camo_0.png and prey_0.txt ... done, elapsed time: 9 seconds.
1/1 [==============================] - 0s 75ms/step
Wrote '0.5130777 0.58160543 0.5358253 0.5782273 0.53510314 0.56456053 ' to response file find_0.txt
  fine_tune_images shape = (1, 128, 128, 3) -- fine_tune_labels shape = (1, 3, 2)
  nearest_center: 50% (nearest_center = 0.5 , steps = 1 )
  -0.392 -0.008 -0.107 -0.968 -0.19 -0.096 0.103 0.009 -0.005 -3.859 0.288 1.045 -0.162 -5.435 -0.021 6.255 0.029 0.21 
  nearest_center: 100% (nearest_center = 1.0 , steps = 1 )
  -0.391 -0.007 

Waiting for camo_7.png and prey_7.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 10ms/step
Wrote '0.26737508 0.465482 0.26605535 0.47508043 0.2543581 0.477519 ' to response file find_7.txt
  fine_tune_images shape = (8, 128, 128, 3) -- fine_tune_labels shape = (8, 3, 2)
  nearest_center: 50% (nearest_center = 4.0 , steps = 8 )
  -0.391 -0.008 -0.107 -0.969 -0.191 -0.095 0.104 0.01 -0.006 -3.859 0.289 1.046 -0.161 -5.436 -0.02 6.254 0.03 0.21 
  nearest_center: 56% (nearest_center = 4.5 , steps = 8 )
  -0.39 -0.009 -0.107 -0.971 -0.192 -0.095 0.103 0.01 -0.005 -3.858 0.29 1.045 -0.163 -5.435 -0.019 6.254 0.028 0.211 
  nearest_center: 62% (nearest_center = 5.0 , steps = 8 )
  -0.392 -0.008 -0.107 -0.969 -0.19 -0.095 0.105 0.008 -0.005 -3.859 0.288 1.045 -0.162 -5.436 -0.021 6.253 0.028 0.212 

step 8

Waiting for camo_8.png and prey_8.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 77ms/step
Wrote '0.63455284 0.5760817 0

  -0.392 -0.008 -0.108 -0.97 -0.192 -0.095 0.104 0.009 -0.005 -3.861 0.29 1.045 -0.163 -5.435 -0.019 6.255 0.027 0.212 

step 16

Waiting for camo_16.png and prey_16.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.27395472 0.77185917 0.25347716 0.7628545 0.26003864 0.75382924 ' to response file find_16.txt
  fine_tune_images shape = (17, 128, 128, 3) -- fine_tune_labels shape = (17, 3, 2)
  nearest_center: 29% (nearest_center = 5.0 , steps = 17 )
  -0.391 -0.009 -0.109 -0.97 -0.191 -0.096 0.104 0.011 -0.007 -3.859 0.29 1.046 -0.16 -5.435 -0.02 6.254 0.027 0.21 
  nearest_center: 29% (nearest_center = 5.0 , steps = 17 )
  -0.392 -0.009 -0.107 -0.97 -0.192 -0.094 0.105 0.008 -0.006 -3.858 0.289 1.047 -0.162 -5.436 -0.02 6.254 0.028 0.211 
  nearest_center: 29% (nearest_center = 5.0 , steps = 17 )
  -0.392 -0.008 -0.108 -0.97 -0.192 -0.095 0.104 0.009 -0.005 -3.861 0.29 1.045 -0.163 -5.435 -0.019 6.255 0.027 0.212 

step 17

Waiting for c

  -0.392 -0.008 -0.107 -0.969 -0.19 -0.095 0.105 0.008 -0.005 -3.859 0.288 1.045 -0.162 -5.436 -0.021 6.253 0.028 0.212 

step 25

Waiting for camo_25.png and prey_25.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/step
Wrote '0.45663357 0.4688955 0.4507897 0.4783682 0.44363832 0.49025545 ' to response file find_25.txt
  fine_tune_images shape = (26, 128, 128, 3) -- fine_tune_labels shape = (26, 3, 2)
  nearest_center: 38% (nearest_center = 10.0 , steps = 26 )
  -0.392 -0.008 -0.107 -0.969 -0.19 -0.094 0.103 0.011 -0.006 -3.859 0.289 1.044 -0.161 -5.436 -0.02 6.254 0.028 0.211 
  nearest_center: 40% (nearest_center = 10.5 , steps = 26 )
  -0.392 -0.008 -0.108 -0.97 -0.192 -0.095 0.104 0.009 -0.005 -3.861 0.29 1.045 -0.163 -5.435 -0.019 6.255 0.027 0.212 
  nearest_center: 42% (nearest_center = 11.0 , steps = 26 )
  -0.392 -0.008 -0.109 -0.971 -0.192 -0.095 0.104 0.01 -0.004 -3.861 0.29 1.045 -0.161 -5.434 -0.018 6.254 0.027 0.21 

step 26

Waiting 

  -0.392 -0.008 -0.107 -0.969 -0.19 -0.095 0.105 0.008 -0.005 -3.859 0.288 1.045 -0.162 -5.436 -0.021 6.253 0.028 0.212 

step 34

Waiting for camo_34.png and prey_34.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 10ms/step
Wrote '0.2376337 0.52770567 0.24498495 0.53700024 0.2407434 0.5418923 ' to response file find_34.txt
  fine_tune_images shape = (35, 128, 128, 3) -- fine_tune_labels shape = (35, 3, 2)
  nearest_center: 57% (nearest_center = 20.0 , steps = 35 )
  -0.391 -0.007 -0.106 -0.968 -0.191 -0.097 0.105 0.009 -0.004 -3.86 0.288 1.045 -0.161 -5.437 -0.019 6.253 0.03 0.212 
  nearest_center: 57% (nearest_center = 20.0 , steps = 35 )
  -0.391 -0.008 -0.107 -0.969 -0.191 -0.095 0.104 0.01 -0.006 -3.859 0.289 1.046 -0.161 -5.436 -0.02 6.254 0.03 0.21 
  nearest_center: 57% (nearest_center = 20.0 , steps = 35 )
  -0.391 -0.009 -0.109 -0.97 -0.191 -0.096 0.104 0.011 -0.007 -3.859 0.29 1.046 -0.16 -5.435 -0.02 6.254 0.027 0.21 

step 35

Waiting for 

...

5/5 [==============================] - 1s 174ms/step - loss: 0.0198 - accuracy: 0.7755 - in_disk: 0.4150
  -0.392 -0.002 -0.109 -0.971 -0.194 -0.088 0.097 0.003 -0.007 -3.859 0.29 1.051 -0.16 -5.435 -0.02 6.258 0.028 0.213 
  nearest_center: 59% (nearest_center = 87.0 , steps = 147 )
Running on CPU ONLY!
5/5 [==============================] - 1s 181ms/step - loss: 0.0146 - accuracy: 0.8299 - in_disk: 0.4490
  -0.394 -0.006 -0.107 -0.978 -0.2 -0.105 0.11 -0.002 -0.005 -3.859 0.288 1.047 -0.162 -5.436 -0.021 6.255 0.025 0.215 
  nearest_center: 59% (nearest_center = 87.0 , steps = 147 )
Running on CPU ONLY!
5/5 [==============================] - 1s 201ms/step - loss: 0.0146 - accuracy: 0.8027 - in_disk: 0.5170
  -0.372 0.024 -0.108 -0.964 -0.186 -0.098 0.105 -0.004 -0.005 -3.859 0.291 1.044 -0.161 -5.437 -0.02 6.26 0.024 0.226 

starving!!   9.125 ,  0.1095890410958904
reinitializing predator 6325544368


step 147

Waiting for camo_147.png and prey_147.txt ... done, elapsed time: 0 secon

5/5 [==============================] - 2s 197ms/step - loss: 0.0194 - accuracy: 0.8105 - in_disk: 0.4314
  -0.391 -0.004 -0.108 -0.972 -0.194 -0.087 0.097 0.005 -0.005 -3.858 0.289 1.051 -0.161 -5.434 -0.019 6.256 0.026 0.216 
  nearest_center: 59% (nearest_center = 91.5 , steps = 153 )
Running on CPU ONLY!
5/5 [==============================] - 1s 195ms/step - loss: 0.0151 - accuracy: 0.8366 - in_disk: 0.4118
  -0.372 0.007 -0.108 -0.966 -0.191 -0.107 0.133 0.0 -0.005 -3.861 0.29 1.052 -0.163 -5.435 -0.019 6.264 0.022 0.223 
  nearest_center: 60% (nearest_center = 92.0 , steps = 153 )
Running on CPU ONLY!
5/5 [==============================] - 1s 185ms/step - loss: 0.0145 - accuracy: 0.8301 - in_disk: 0.4314
  -0.38 0.014 -0.105 -0.963 -0.198 -0.101 0.113 0.021 -0.005 -3.859 0.289 1.043 -0.163 -5.435 -0.018 6.259 0.021 0.217 

starving!!   8.941176470588236 ,  0.1118421052631579
reinitializing predator 6342023872


step 153

Waiting for camo_153.png and prey_153.txt ... done, elapsed 

5/5 [==============================] - 1s 198ms/step - loss: 0.0123 - accuracy: 0.8302 - in_disk: 0.4843
  -0.374 -0.0 -0.106 -0.962 -0.192 -0.087 0.103 0.003 -0.006 -3.86 0.291 1.041 -0.163 -5.435 -0.019 6.258 0.022 0.22 
  nearest_center: 59% (nearest_center = 94.5 , steps = 159 )
Running on CPU ONLY!
5/5 [==============================] - 1s 202ms/step - loss: 0.0136 - accuracy: 0.8113 - in_disk: 0.5094
  -0.391 0.018 -0.109 -0.957 -0.195 -0.107 0.117 0.005 -0.003 -3.86 0.292 1.041 -0.164 -5.433 -0.019 6.254 0.021 0.219 
  nearest_center: 59% (nearest_center = 94.5 , steps = 159 )
Running on CPU ONLY!
5/5 [==============================] - 1s 194ms/step - loss: 0.0138 - accuracy: 0.8239 - in_disk: 0.4214
  -0.382 0.001 -0.107 -0.969 -0.182 -0.107 0.12 0.001 -0.006 -3.858 0.289 1.055 -0.162 -5.436 -0.02 6.255 0.022 0.22 

step 159

Waiting for camo_159.png and prey_159.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 8ms/step
Wrote '0.76106894 0.474539

6/6 [==============================] - 1s 171ms/step - loss: 0.0142 - accuracy: 0.8788 - in_disk: 0.5091
  -0.395 -0.006 -0.109 -0.964 -0.191 -0.085 0.097 0.004 -0.007 -3.859 0.29 1.056 -0.16 -5.435 -0.02 6.259 0.025 0.218 
  nearest_center: 58% (nearest_center = 96.5 , steps = 165 )
Running on CPU ONLY!
6/6 [==============================] - 1s 163ms/step - loss: 0.0173 - accuracy: 0.7818 - in_disk: 0.4667
  -0.392 0.018 -0.109 -0.956 -0.196 -0.108 0.117 0.003 -0.003 -3.86 0.292 1.04 -0.164 -5.433 -0.019 6.256 0.022 0.22 
  nearest_center: 58% (nearest_center = 97.0 , steps = 165 )
Running on CPU ONLY!
6/6 [==============================] - 1s 168ms/step - loss: 0.0147 - accuracy: 0.8242 - in_disk: 0.4909
  -0.383 0.001 -0.107 -0.969 -0.184 -0.107 0.119 -0.001 -0.006 -3.858 0.289 1.055 -0.162 -5.436 -0.02 6.255 0.022 0.22 

step 165

Waiting for camo_165.png and prey_165.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
Wrote '0.4805377 0.351535

KeyboardInterrupt: 

In [ ]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.

# restart_run()

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#123456789 123456789 123456789 123456789 123456789 123456789 123456789 123456789